In [7]:
import psycopg2
from dotenv import load_dotenv

load_dotenv()

import os

def delete_all_bot_logs():
    connection = psycopg2.connect(
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT"),
        database=os.getenv("DB_DATABASE")
    )
    try:
        cursor = connection.cursor()
        cursor.execute("DELETE FROM bot_logs;")
        connection.commit()
        print("All rows in the bot_logs table have been deleted.")
    except Exception as error:
        print(f"An error occurred: {error}")
    finally:
        cursor.close()
        connection.close()
        
        
def check_db_entries():
    connection = psycopg2.connect(
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT"),
        database=os.getenv("DB_DATABASE")
    )
    try:
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM bot_logs;")
        #cursor.execute(f"INSERT INTO MAILING_HISTORY (DATE) VALUES ('{current_date}');")
        records = cursor.fetchall()
        # for record in records:
        #     print(record)
        connection.commit()
        return records
    except Exception as error:
        print(f"An error occurred: {error}")
    finally:
        cursor.close()
        connection.close()

check_db_entries()


[]

In [5]:
# from db_interaction import *

# create_tables_DB()

docker run -d --name jkuclassnotifier --restart=always -e TZ="CET" jkuclassnotifier

In [17]:
from db_interaction import *

get_all_users_DB()

[(547495974,
  'https://www.kusss.jku.at/kusss/published-calendar.action?token=aJSCi178m3-9rN6qZ1qt5zxxM_4baG2XDmaiXccN&lang=en',
  0,
  0),
 (1194532650,
  'https://www.kusss.jku.at/kusss/published-calendar.action?token=M1ckSno0_TJR5tASv2U8jUPDLIC9csKhwEZLBe58&lang=en',
  0,
  0),
 (7010939891,
  'https://www.kusss.jku.at/kusss/published-calendar.action?token=SnqTtuo3-SALSSUQhKTYyu8-jGk47kQCHO7AcRWo&lang=en',
  0,
  0),
 (7760135403,
  'https://www.kusss.jku.at/kusss/published-calendar.action?token=igwGoYzotdLRNGTzQWI4yXWcxQTlgwZ3xUbgcvMx&lang=en',
  0,
  0),
 (5303965494,
  'https://www.kusss.jku.at/kusss/published-calendar.action?token=yfqJ0X1_NO87gomi3JMOBuMNPaDIMOSEZNoQVfYt&lang=en',
  2,
  27)]